# Prection of live feed

This code will utiliz the custom trainede yolo8 modle to do predictions on a live feed

In [1]:
# Importing the required libraries
import cv2
from ultralytics import YOLO

c:\Users\lau21\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Load your locally trained YOLOv8 model
model = YOLO("runs/detect/train/weights/best.pt")  # Update path if needed

In [3]:
# Define the specific classes you want to detect
target_classes = ["1", "2", "3","4","5","6"]  # Match your training labels

In [4]:
# Validate classes
available_classes = list(model.names.values())
valid_classes = [cls for cls in target_classes if cls in available_classes]

if not valid_classes:
    raise ValueError(f"Error: Target classes not found in model. Available: {available_classes}")

In [5]:
# Get class indices
class_ids = [list(model.names.keys())[list(model.names.values()).index(cls)] for cls in valid_classes]

In [6]:
# Set minimum confidence threshold
CONFIDENCE_THRESHOLD = 0.5  # Adjust this value as needed (0.0 - 1.0)

In [7]:
#  This Python function uses OpenCV and a YOLOv8 model to perform real-time object detection from a webcam feed, 
#  specifically for detecting and counting fasteners (or other predefined object classes).

def live_fastener_detection():

    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to capture frame.")
                break

            # Run detection on the frame
            results = model(frame, classes=class_ids, verbose=False)

            # Get result for this frame
            boxes = results[0].boxes
            annotated_frame = frame.copy()

            # Count detections
            counts = {cls: 0 for cls in valid_classes}

            for box in boxes:
                conf = float(box.conf)
                if conf < CONFIDENCE_THRESHOLD:
                    continue  # Skip low-confidence detections

                cls_id = int(box.cls)
                cls_name = model.names[cls_id]
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Draw bounding box
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(annotated_frame, f"{cls_name} ({conf:.2f})", 
                            (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                            0.5, (0, 255, 255), 2)

                # Count
                if cls_name in counts:
                    counts[cls_name] += 1

            # Draw detection counts
            y_offset = 30
            for cls, count in counts.items():
                cv2.putText(annotated_frame, f"{cls}: {count}",
                            (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (255, 0, 0), 2)
                y_offset += 30

            # Show annotated frame
            cv2.imshow("Local YOLOv8 Detection", annotated_frame)

            # Exit on 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    live_fastener_detection()